# MCP (Model Context Protocol)


이 워크샵에서는 Agent Framework에서 MCP(Model Context Protocol)를 사용하는 방법을 학습합니다.

## 목차
1. [MCP란 무엇인가?](#mcp란-무엇인가)
2. [환경 설정](#환경-설정)
3. [Agent를 MCP Server로 노출하기](#agent를-mcp-server로-노출하기)
4. [MCP 인증 (API Key)](#mcp-인증-api-key)
5. [MCP 도구 사용하기](#mcp-도구-사용하기)
6. [실전 예제](#실전-예제)

## 학습 목표
- MCP의 개념과 작동 방식 이해
- Agent를 MCP Server로 노출하는 방법 학습
- MCP 인증 방법 이해
- MCP 도구를 Agent에 통합하는 방법 학습

## 1. MCP란 무엇인가?

**Model Context Protocol (MCP)**는 AI 에이전트를 데이터 소스 및 도구와 연결하기 위한 개방형 표준입니다.

### 주요 특징

- 🔌 **표준화된 연결**: 로컬 및 원격 리소스에 대한 표준화된 프로토콜
- 🔐 **보안**: 안전하고 통제된 리소스 액세스
- 🔄 **상호 운용성**: 다양한 AI 애플리케이션 간 연동
- 🛠️ **도구 노출**: Agent의 기능을 다른 시스템에 노출

### MCP의 주요 구성 요소

1. **MCP Server**: 도구와 리소스를 제공하는 서버
2. **MCP Client**: 서버에 연결하여 도구를 사용하는 클라이언트
3. **Tools**: 서버가 제공하는 기능
4. **Resources**: 서버가 제공하는 데이터

### 사용 사례

- Claude Desktop에 커스텀 도구 추가
- VS Code Copilot에 도구 통합
- 여러 AI 시스템 간 도구 공유
- 로컬 파일 시스템, 데이터베이스 등에 안전하게 액세스

## 2. 환경 설정

필요한 패키지를 설치하고 환경 변수를 설정합니다.

In [1]:
# 필요한 패키지 설치
# !pip install agent-framework agent-framework-azure-ai mcp python-dotenv

In [2]:
# 환경 변수 로드
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드 (override=True로 기존 값 덮어쓰기)
load_dotenv(override=True)

# 필요한 환경 변수 확인
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME",
]

print("환경 변수 확인:")
for var in required_vars:
    status = "✓" if os.getenv(var) else "✗"
    value = os.getenv(var, "")
    # API 키는 마스킹해서 표시
    if "API_KEY" in var and value:
        display_value = f"{value[:3]}...{value[-3:]}"
    elif value:
        display_value = value
    else:
        display_value = "Not set"
    print(f"{status} {var}: {display_value}")

환경 변수 확인:
✓ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✓ AZURE_OPENAI_API_KEY: 6hB...Teh
✓ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1


## 3. Agent를 MCP Server로 노출하기

Agent Framework의 Agent를 MCP Server로 노출하여 다른 AI 애플리케이션에서 사용할 수 있도록 만들어봅시다.

### 3.1 레스토랑 메뉴 Agent 만들기

In [3]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient

# 레스토랑 메뉴 관련 도구 정의
def get_specials() -> Annotated[str, "Returns the specials from the menu."]:
    """오늘의 특선 메뉴를 반환합니다."""
    return """
        Special Soup: Clam Chowder (조개 차우더)
        Special Salad: Cobb Salad (콥 샐러드)
        Special Drink: Chai Tea (차이 티)
        """


def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    """메뉴 아이템의 가격을 반환합니다."""
    # 실제로는 데이터베이스에서 조회하겠지만, 여기서는 간단히 고정값 반환
    prices = {
        "clam chowder": "$12.99",
        "cobb salad": "$14.99",
        "chai tea": "$4.99",
        "burger": "$15.99",
        "pasta": "$16.99",
        "steak": "$29.99",
    }
    
    item_lower = menu_item.lower()
    for key, price in prices.items():
        if key in item_lower:
            return f"{menu_item}: {price}"
    
    return f"{menu_item}: $9.99 (기본 가격)"


print("✓ 레스토랑 도구 정의 완료")

✓ 레스토랑 도구 정의 완료


### 3.2 Agent 생성 및 테스트

In [4]:
# Agent 생성
restaurant_agent = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version="2024-10-21",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
).create_agent(
    name="RestaurantAgent",
    description="Answer questions about the menu.",
    instructions="You are a helpful restaurant assistant. Answer questions about the menu and specials.",
    tools=[get_specials, get_item_price],
)

print("✓ RestaurantAgent 생성 완료")
print(f"Agent 이름: {restaurant_agent.name}")
print(f"Agent 설명: {restaurant_agent.description}")

✓ RestaurantAgent 생성 완료
Agent 이름: RestaurantAgent
Agent 설명: Answer questions about the menu.


In [5]:
# Agent 테스트
async def test_restaurant_agent():
    """레스토랑 Agent 테스트"""
    print("=== Restaurant Agent 테스트 ===")
    
    # 특선 메뉴 조회
    query1 = "What are today's specials?"
    print(f"\n[사용자] {query1}")
    result1 = await restaurant_agent.run(query1)
    print(f"[Agent] {result1.text}")
    
    # 가격 조회
    query2 = "How much is the Cobb Salad?"
    print(f"\n[사용자] {query2}")
    result2 = await restaurant_agent.run(query2)
    print(f"[Agent] {result2.text}")
    
    # 복합 질문
    query3 = "What are the specials and how much do they cost?"
    print(f"\n[사용자] {query3}")
    result3 = await restaurant_agent.run(query3)
    print(f"[Agent] {result3.text}")

await test_restaurant_agent()

=== Restaurant Agent 테스트 ===

[사용자] What are today's specials?
[Agent] Today's specials are:

- Special Soup: Clam Chowder (조개 차우더)
- Special Salad: Cobb Salad (콥 샐러드)
- Special Drink: Chai Tea (차이 티)

Would you like to know more about any of these specials or their prices?

[사용자] How much is the Cobb Salad?
[Agent] Today's specials are:

- Special Soup: Clam Chowder (조개 차우더)
- Special Salad: Cobb Salad (콥 샐러드)
- Special Drink: Chai Tea (차이 티)

Would you like to know more about any of these specials or their prices?

[사용자] How much is the Cobb Salad?
[Agent] The Cobb Salad is $14.99. If you have any other questions about the menu, feel free to ask!

[사용자] What are the specials and how much do they cost?
[Agent] The Cobb Salad is $14.99. If you have any other questions about the menu, feel free to ask!

[사용자] What are the specials and how much do they cost?
[Agent] Today's specials are:

- Special Soup: Clam Chowder - $12.99
- Special Salad: Cobb Salad - $14.99
- Special Drink: Chai Tea

### 3.3 Agent를 MCP Server로 노출하기

Agent를 MCP Server로 노출하면 Claude Desktop, VS Code Copilot 등 다른 AI 애플리케이션에서 사용할 수 있습니다.

#### MCP Server 설정 방법

**Claude Desktop 설정 예시** (`~/Library/Application Support/Claude/claude_desktop_config.json`):

```json
{
    "mcpServers": {
        "agent-framework": {
            "command": "uv",
            "args": [
                "--directory=/path/to/agent-framework/python/samples/getting_started/mcp",
                "run",
                "agent_as_mcp_server.py"
            ],
            "env": {
                "AZURE_OPENAI_ENDPOINT": "https://your-resource.openai.azure.com",
                "AZURE_OPENAI_API_KEY": "your-api-key",
                "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME": "gpt-4"
            }
        }
    }
}
```

**VS Code Copilot 설정 예시** (`.vscode/settings.json`):

```json
{
    "github.copilot.chat.mcp.servers": {
        "agent-framework": {
            "command": "uv",
            "args": [
                "--directory=/path/to/agent-framework/python/samples/getting_started/mcp",
                "run",
                "agent_as_mcp_server.py"
            ],
            "env": {
                "AZURE_OPENAI_ENDPOINT": "https://your-resource.openai.azure.com",
                "AZURE_OPENAI_API_KEY": "your-api-key",
                "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME": "gpt-4"
            }
        }
    }
}
```

In [6]:
# MCP Server 생성 (주피터 노트북에서는 실제로 실행하지 않음)
# 실제 사용 시에는 별도 Python 스크립트로 실행해야 합니다

def create_mcp_server_code():
    """MCP Server 코드 생성 예제"""
    code = '''
# agent_as_mcp_server.py
import os
import anyio
from typing import Annotated, Any
from agent_framework.azure import AzureOpenAIChatClient

def get_specials() -> Annotated[str, "Returns the specials from the menu."]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    return "$9.99"

async def run() -> None:
    # Agent 정의
    agent = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ).create_agent(
        name="RestaurantAgent",
        description="Answer questions about the menu.",
        tools=[get_specials, get_item_price],
    )

    # Agent를 MCP Server로 노출
    server = agent.as_mcp_server()

    # Server 실행
    from mcp.server.stdio import stdio_server

    async def handle_stdin(stdin: Any | None = None, stdout: Any | None = None) -> None:
        async with stdio_server() as (read_stream, write_stream):
            await server.run(read_stream, write_stream, server.create_initialization_options())

    await handle_stdin()

if __name__ == "__main__":
    anyio.run(run)
'''
    return code

print("=== MCP Server 코드 예제 ===")
print(create_mcp_server_code())
print("\n💡 이 코드를 별도 Python 파일로 저장하여 실행하세요.")

=== MCP Server 코드 예제 ===

# agent_as_mcp_server.py
import os
import anyio
from typing import Annotated, Any
from agent_framework.azure import AzureOpenAIChatClient

def get_specials() -> Annotated[str, "Returns the specials from the menu."]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    return "$9.99"

async def run() -> None:
    # Agent 정의
    agent = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ).create_agent(
        name="RestaurantAgent",
        description="Answer questions about the menu.",
        tools=[get_specials, get_item_price],
    )

    # Agent를 MCP Server

## 4. MCP 인증 (API Key)

MCP Server에 연결할 때 API Key를 사용한 인증 방법을 학습합니다.

### 4.1 인증 방식

MCP는 다양한 인증 방식을 지원합니다:

1. **Bearer Token**: `Authorization: Bearer <token>`
2. **API Key Header**: `X-API-Key: <key>`
3. **Custom Header**: `Authorization: ApiKey <key>`
4. **OAuth 2.0**: OAuth 2.0 플로우 사용

### 4.2 API Key 인증 예제

In [7]:
from agent_framework import ChatAgent, MCPStreamableHTTPTool

# 인증 헤더 생성 함수
def create_auth_headers(api_key: str, auth_type: str = "bearer") -> dict[str, str]:
    """
    MCP 인증 헤더 생성
    
    Args:
        api_key: API 키
        auth_type: 인증 타입 ('bearer', 'api_key', 'custom')
    
    Returns:
        인증 헤더 딕셔너리
    """
    if auth_type == "bearer":
        return {"Authorization": f"Bearer {api_key}"}
    elif auth_type == "api_key":
        return {"X-API-Key": api_key}
    elif auth_type == "custom":
        return {"Authorization": f"ApiKey {api_key}"}
    else:
        raise ValueError(f"Unknown auth_type: {auth_type}")


# 인증 헤더 예시
example_key = "your-api-key-here"
print("=== 인증 헤더 예시 ===")
print(f"Bearer Token: {create_auth_headers(example_key, 'bearer')}")
print(f"API Key Header: {create_auth_headers(example_key, 'api_key')}")
print(f"Custom Header: {create_auth_headers(example_key, 'custom')}")

=== 인증 헤더 예시 ===
Bearer Token: {'Authorization': 'Bearer your-api-key-here'}
API Key Header: {'X-API-Key': 'your-api-key-here'}
Custom Header: {'Authorization': 'ApiKey your-api-key-here'}


### 4.3 인증된 MCP 도구 사용하기

실제 MCP Server에 인증하여 연결하는 예제입니다.

In [8]:
async def api_key_auth_example():
    """
    API Key 인증을 사용한 MCP Server 연결 예제
    
    실제 사용 시에는:
    1. MCP_SERVER_URL 환경 변수에 MCP Server URL 설정
    2. MCP_API_KEY 환경 변수에 API Key 설정
    """
    # 환경 변수에서 설정 로드
    mcp_server_url = os.getenv("MCP_SERVER_URL")
    mcp_api_key = os.getenv("MCP_API_KEY")
    
    if not mcp_server_url or not mcp_api_key:
        print("⚠️  MCP_SERVER_URL 또는 MCP_API_KEY가 설정되지 않았습니다.")
        print("데모 코드만 표시합니다.\n")
        
        demo_code = '''
# 인증 헤더 생성
auth_headers = {
    "Authorization": f"Bearer {mcp_api_key}",
}

# Azure OpenAI 클라이언트 생성
client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version="2024-10-21",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# MCP 도구 생성 (인증 포함)
async with (
    MCPStreamableHTTPTool(
        name="MCP tool",
        description="MCP tool description",
        url=mcp_server_url,
        headers=auth_headers,  # 인증 헤더
    ) as mcp_tool,
    ChatAgent(
        chat_client=client,
        name="Agent",
        instructions="You are a helpful assistant.",
        tools=mcp_tool,
    ) as agent,
):
    query = "What tools are available to you?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"Agent: {result.text}")
'''
        print(demo_code)
        return
    
    # 실제 연결 시도
    print(f"MCP Server URL: {mcp_server_url}")
    print("연결 중...\n")
    
    auth_headers = {"Authorization": f"Bearer {mcp_api_key}"}
    
    # Azure OpenAI 클라이언트 생성
    client = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    )
    
    try:
        async with (
            MCPStreamableHTTPTool(
                name="MCP tool",
                description="MCP tool description",
                url=mcp_server_url,
                headers=auth_headers,
            ) as mcp_tool,
            ChatAgent(
                chat_client=client,
                name="Agent",
                instructions="You are a helpful assistant.",
                tools=mcp_tool,
            ) as agent,
        ):
            query = "What tools are available to you?"
            print(f"[사용자] {query}")
            result = await agent.run(query)
            print(f"[Agent] {result.text}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

await api_key_auth_example()

⚠️  MCP_SERVER_URL 또는 MCP_API_KEY가 설정되지 않았습니다.
데모 코드만 표시합니다.


# 인증 헤더 생성
auth_headers = {
    "Authorization": f"Bearer {mcp_api_key}",
}

# Azure OpenAI 클라이언트 생성
client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version="2024-10-21",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# MCP 도구 생성 (인증 포함)
async with (
    MCPStreamableHTTPTool(
        name="MCP tool",
        description="MCP tool description",
        url=mcp_server_url,
        headers=auth_headers,  # 인증 헤더
    ) as mcp_tool,
    ChatAgent(
        chat_client=client,
        name="Agent",
        instructions="You are a helpful assistant.",
        tools=mcp_tool,
    ) as agent,
):
    query = "What tools are available to you?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"Agent: {result.text}")



## 5. MCP 도구 사용하기

MCP를 통해 외부 도구를 Agent에 통합하는 방법을 학습합니다.

### 5.1 MCPStreamableHTTPTool 개요

`MCPStreamableHTTPTool`은 HTTP를 통해 MCP Server에 연결하는 도구입니다.

**주요 파라미터**:
- `name`: 도구 이름
- `description`: 도구 설명
- `url`: MCP Server URL
- `headers`: 인증 헤더 (선택사항)

### 5.2 MCP 도구 통합 패턴

In [9]:
# MCP 도구 통합 패턴 예제

def show_mcp_integration_pattern():
    """MCP 도구 통합 패턴 설명"""
    pattern = '''
=== MCP 도구 통합 패턴 ===

1. 기본 패턴 (인증 없음)
   ────────────────────
   async with (
       MCPStreamableHTTPTool(
           name="Tool Name",
           description="Tool Description",
           url="https://mcp-server.example.com",
       ) as mcp_tool,
       ChatAgent(
           chat_client=client,
           tools=mcp_tool,
       ) as agent,
   ):
       result = await agent.run("Your query")

2. 인증 패턴 (API Key)
   ────────────────────
   headers = {"Authorization": f"Bearer {api_key}"}
   
   async with (
       MCPStreamableHTTPTool(
           name="Tool Name",
           description="Tool Description",
           url="https://mcp-server.example.com",
           headers=headers,  # 인증 헤더 추가
       ) as mcp_tool,
       ChatAgent(
           chat_client=client,
           tools=mcp_tool,
       ) as agent,
   ):
       result = await agent.run("Your query")

3. 여러 MCP 도구 통합
   ────────────────────
   async with (
       MCPStreamableHTTPTool(...) as mcp_tool1,
       MCPStreamableHTTPTool(...) as mcp_tool2,
       ChatAgent(
           chat_client=client,
           tools=[mcp_tool1, mcp_tool2],  # 여러 도구
       ) as agent,
   ):
       result = await agent.run("Your query")

4. MCP + 로컬 도구 혼합
   ────────────────────
   def local_tool(param: str) -> str:
       return "result"
   
   async with (
       MCPStreamableHTTPTool(...) as mcp_tool,
       ChatAgent(
           chat_client=client,
           tools=[mcp_tool, local_tool],  # MCP + 로컬
       ) as agent,
   ):
       result = await agent.run("Your query")
'''
    print(pattern)

show_mcp_integration_pattern()


=== MCP 도구 통합 패턴 ===

1. 기본 패턴 (인증 없음)
   ────────────────────
   async with (
       MCPStreamableHTTPTool(
           name="Tool Name",
           description="Tool Description",
           url="https://mcp-server.example.com",
       ) as mcp_tool,
       ChatAgent(
           chat_client=client,
           tools=mcp_tool,
       ) as agent,
   ):
       result = await agent.run("Your query")

2. 인증 패턴 (API Key)
   ────────────────────
   headers = {"Authorization": f"Bearer {api_key}"}

   async with (
       MCPStreamableHTTPTool(
           name="Tool Name",
           description="Tool Description",
           url="https://mcp-server.example.com",
           headers=headers,  # 인증 헤더 추가
       ) as mcp_tool,
       ChatAgent(
           chat_client=client,
           tools=mcp_tool,
       ) as agent,
   ):
       result = await agent.run("Your query")

3. 여러 MCP 도구 통합
   ────────────────────
   async with (
       MCPStreamableHTTPTool(...) as mcp_tool1,
       MCPStreamableHT

## 6. 실전 예제

실제 사용 사례를 바탕으로 한 종합 예제입니다.

### 6.1 시나리오: 다국어 번역 서비스

MCP Server로 번역 서비스를 제공하는 Agent를 만들어봅시다.

In [10]:
from typing import Annotated

# 번역 도구 정의
def translate_text(
    text: Annotated[str, "The text to translate"],
    target_language: Annotated[str, "Target language (e.g., 'ko', 'en', 'ja', 'zh')"],
    source_language: Annotated[str, "Source language (auto-detect if not specified)"] = "auto",
) -> Annotated[str, "Translated text"]:
    """
    텍스트를 지정된 언어로 번역합니다.
    
    실제로는 번역 API를 호출하겠지만, 여기서는 시뮬레이션합니다.
    """
    translations = {
        ("Hello, how are you?", "ko"): "안녕하세요, 어떻게 지내세요?",
        ("Thank you very much", "ko"): "정말 감사합니다",
        ("Good morning", "ko"): "좋은 아침입니다",
        ("안녕하세요", "en"): "Hello",
        ("감사합니다", "en"): "Thank you",
    }
    
    key = (text, target_language)
    if key in translations:
        return translations[key]
    
    return f"[Simulated translation of '{text}' to {target_language}]"


def get_supported_languages() -> Annotated[str, "List of supported languages"]:
    """지원하는 언어 목록을 반환합니다."""
    return """
    Supported Languages:
    - Korean (ko)
    - English (en)
    - Japanese (ja)
    - Chinese (zh)
    - Spanish (es)
    - French (fr)
    - German (de)
    """


print("✓ 번역 도구 정의 완료")

✓ 번역 도구 정의 완료


In [11]:
# 번역 Agent 생성 및 테스트
async def test_translation_agent():
    """번역 Agent 테스트"""
    print("=== Translation Agent 예제 ===")
    
    translation_agent = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ).create_agent(
        name="TranslationAgent",
        description="Translate text between different languages.",
        instructions=(
            "You are a helpful translation assistant. "
            "Use the available tools to translate text and check supported languages."
        ),
        tools=[translate_text, get_supported_languages],
    )
    
    # 테스트 쿼리
    queries = [
        "What languages do you support?",
        "Translate 'Hello, how are you?' to Korean",
        "Translate '안녕하세요' to English",
    ]
    
    for query in queries:
        print(f"\n[사용자] {query}")
        result = await translation_agent.run(query)
        print(f"[Agent] {result.text}")

await test_translation_agent()

=== Translation Agent 예제 ===

[사용자] What languages do you support?
[Agent] I support the following languages for translation:

- Korean (ko)
- English (en)
- Japanese (ja)
- Chinese (zh)
- Spanish (es)
- French (fr)
- German (de)

If you need translation to or from any of these languages, let me know!

[사용자] Translate 'Hello, how are you?' to Korean
[Agent] I support the following languages for translation:

- Korean (ko)
- English (en)
- Japanese (ja)
- Chinese (zh)
- Spanish (es)
- French (fr)
- German (de)

If you need translation to or from any of these languages, let me know!

[사용자] Translate 'Hello, how are you?' to Korean
[Agent] "Hello, how are you?" in Korean is: 안녕하세요, 어떻게 지내세요?

[사용자] Translate '안녕하세요' to English
[Agent] "Hello, how are you?" in Korean is: 안녕하세요, 어떻게 지내세요?

[사용자] Translate '안녕하세요' to English
[Agent] '안녕하세요' translates to "Hello" in English.
[Agent] '안녕하세요' translates to "Hello" in English.


### 6.2 시나리오: 날씨 정보 서비스

날씨 정보를 제공하는 MCP Agent를 만들어봅시다.

In [12]:
from datetime import datetime

# 날씨 도구 정의
def get_current_weather(
    city: Annotated[str, "City name (e.g., 'Seoul', 'Tokyo', 'New York')"],
) -> Annotated[str, "Current weather information"]:
    """지정된 도시의 현재 날씨를 반환합니다."""
    # 실제로는 날씨 API를 호출하겠지만, 여기서는 시뮬레이션
    weather_data = {
        "seoul": {"temp": 18, "condition": "맑음", "humidity": 60},
        "tokyo": {"temp": 22, "condition": "구름 조금", "humidity": 65},
        "new york": {"temp": 15, "condition": "비", "humidity": 75},
        "london": {"temp": 12, "condition": "흐림", "humidity": 80},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        return (
            f"{city} 날씨:\n"
            f"- 온도: {data['temp']}°C\n"
            f"- 상태: {data['condition']}\n"
            f"- 습도: {data['humidity']}%\n"
            f"- 조회 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        )
    
    return f"{city}의 날씨 정보를 찾을 수 없습니다."


def get_weather_forecast(
    city: Annotated[str, "City name"],
    days: Annotated[int, "Number of days (1-7)"] = 3,
) -> Annotated[str, "Weather forecast"]:
    """지정된 도시의 날씨 예보를 반환합니다."""
    return f"{city}의 {days}일 날씨 예보:\n" + "\n".join(
        [f"Day {i+1}: 맑음, 온도 {20+i}°C" for i in range(days)]
    )


print("✓ 날씨 도구 정의 완료")

✓ 날씨 도구 정의 완료


In [13]:
# 날씨 Agent 생성 및 테스트
async def test_weather_agent():
    """날씨 Agent 테스트"""
    print("=== Weather Agent 예제 ===")
    
    weather_agent = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ).create_agent(
        name="WeatherAgent",
        description="Provide weather information for cities.",
        instructions=(
            "You are a helpful weather assistant. "
            "Use the available tools to provide current weather and forecasts."
        ),
        tools=[get_current_weather, get_weather_forecast],
    )
    
    # 테스트 쿼리
    queries = [
        "What's the weather in Seoul?",
        "Give me a 5-day forecast for Tokyo",
        "Compare the weather in Seoul and London",
    ]
    
    for query in queries:
        print(f"\n[사용자] {query}")
        result = await weather_agent.run(query)
        print(f"[Agent] {result.text}")

await test_weather_agent()

=== Weather Agent 예제 ===

[사용자] What's the weather in Seoul?
[Agent] The current weather in Seoul is clear with a temperature of 18°C and a humidity level of 60%.

[사용자] Give me a 5-day forecast for Tokyo
[Agent] The current weather in Seoul is clear with a temperature of 18°C and a humidity level of 60%.

[사용자] Give me a 5-day forecast for Tokyo
[Agent] Here is the 5-day weather forecast for Tokyo:

- Day 1: Sunny, 20°C
- Day 2: Sunny, 21°C
- Day 3: Sunny, 22°C
- Day 4: Sunny, 23°C
- Day 5: Sunny, 24°C

You can expect clear and gradually warming weather in Tokyo for the next five days!

[사용자] Compare the weather in Seoul and London
[Agent] Here is the 5-day weather forecast for Tokyo:

- Day 1: Sunny, 20°C
- Day 2: Sunny, 21°C
- Day 3: Sunny, 22°C
- Day 4: Sunny, 23°C
- Day 5: Sunny, 24°C

You can expect clear and gradually warming weather in Tokyo for the next five days!

[사용자] Compare the weather in Seoul and London
[Agent] Here is the current weather comparison between Seoul and Lo

## 요약 및 베스트 프랙티스

### MCP 핵심 개념

1. **MCP Server**: 도구와 리소스를 제공하는 표준화된 서버
2. **MCP Client**: 서버에 연결하여 도구를 사용하는 클라이언트
3. **Agent as Server**: Agent를 MCP Server로 노출하여 재사용
4. **인증**: API Key, Bearer Token 등 다양한 인증 방식 지원

### 베스트 프랙티스

#### 1. Agent 설계
```python
# ✓ Good: 명확한 이름과 설명
agent = client.create_agent(
    name="RestaurantAgent",
    description="Answer questions about the menu.",
    tools=[get_specials, get_item_price],
)

# ✗ Bad: 불명확한 이름
agent = client.create_agent(
    name="Agent1",
    description="Does stuff.",
    tools=[func1, func2],
)
```

#### 2. 도구 정의
```python
# ✓ Good: 타입 힌트와 설명 포함
def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    return f"${price}"

# ✗ Bad: 타입 힌트와 설명 없음
def get_item_price(menu_item):
    return price
```

#### 3. 인증 처리
```python
# ✓ Good: 환경 변수에서 API Key 로드
api_key = os.getenv("MCP_API_KEY")
headers = {"Authorization": f"Bearer {api_key}"}

# ✗ Bad: 코드에 하드코딩
headers = {"Authorization": "Bearer abc123"}
```

#### 4. 리소스 관리
```python
# ✓ Good: async with 사용
async with MCPStreamableHTTPTool(...) as mcp_tool:
    result = await agent.run(query)

# ✗ Bad: 수동 관리
mcp_tool = MCPStreamableHTTPTool(...)
result = await agent.run(query)
# 정리 코드 누락
```

### 다음 단계

1. **실제 MCP Server 구축**: `agent.as_mcp_server()`를 사용하여 프로덕션 서버 구축
2. **Claude Desktop 통합**: Claude Desktop에 MCP Server 연결
3. **VS Code Copilot 통합**: VS Code에 MCP Server 추가
4. **OAuth 2.0 인증**: 고급 인증 플로우 구현
5. **멀티 에이전트 시스템**: 여러 MCP Agent를 조합한 시스템 구축

### 참고 자료

- [MCP 공식 문서](https://modelcontextprotocol.io/)
- [Agent Framework 문서](https://github.com/microsoft/agent-framework)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP 인증 예제](https://github.com/modelcontextprotocol/python-sdk/tree/main/examples)